# Анализ РНК

### 1. Выравнивание ридов RNA-Seq

In [ ]:
! /Molly/chernikova/soft/STAR --runThreadN 20 --runMode genomeGenerate --genomeDir genomeDir/ --genomeFastaFiles /Johnny/students/NGS/data/8/ref.fa --sjdbGTFfile /Johnny/students/NGS/data/8/genes.gtf

In [ ]:
! /Molly/chernikova/soft/STAR --runThreadN 20 --genomeDir genomeDir/ --readFilesIn /Johnny/students/NGS/data/8/SRR453566_1.fastq /Johnny/students/NGS/data/8/SRR453566_2.fastq

### 2. Оценка количества экспрессирующихся генов. 

In [2]:
import pysam
import matplotlib.pyplot as plt
%matplotlib inline
from Bio import SeqIO
from BCBio import GFF
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
import pandas as pd

In [3]:
def coverByRead(record, refCover):
    if (record.is_unmapped):
        return
    rpos = record.reference_start
    cigar = record.cigartuples
    rname = record.reference_name
    
    for i in range(len(cigar)):
        if (cigar[i][0] == 0 or cigar[i][0] == 7 or cigar[i][0] == 8):
            for j in range(cigar[i][1]):
                refCover[rname][rpos + j] = 1
            rpos += cigar[i][1]
        elif (cigar[i][0] == 2 or cigar[i][0] == 3):
            rpos += cigar[i][1]

In [4]:
def calcRefCover(refFile, samfile):
    refCover = dict()
    for rec in SeqIO.parse(refFile, "fasta"):
        refCover[rec.id] = [0]*len(rec.seq)
    sam = pysam.AlignmentFile(samfile, "r")
    
    for record in sam.fetch():
        coverByRead(record, refCover)
    return refCover

In [5]:
refCover = calcRefCover("/Johnny/students/NGS/data/8/ref.fa", "Aligned.out.sam")

In [39]:
def main(gtfFile):
    cntGene = 0
    cntCoverGene = 0
    handle = open(gtfFile)
    lines = handle.readlines()
    
    exons = dict()
    for line in lines:
        if "\texon\t" in line:
            spl = line.split('\t')
            bg = spl[3]
            ed = spl[4]
            chrm = spl[0]
            geneid = spl[-1].split(';')[0][9:-1]
            if (geneid not in exons):
                exons[geneid] = []
            exons[geneid].append((chrm, int(bg), int(ed)))
    for key in exons:
        cntGene += 1
        cntpos = 0
        cntCoverPos = 0
        for ex in exons[key]:
            for i in range(ex[1], ex[2]):
                cntpos += 1
                if (refCover[ex[0]][i]):
                    cntCoverPos += 1
        if (cntCoverPos * 100 > 95 * cntpos):
            cntCoverGene += 1
    
    handle.close()
    print(cntCoverGene, cntGene, cntCoverGene/cntGene)

In [40]:
main("/Johnny/students/NGS/data/8/genes.gtf")

6273 7126 0.8802975021049677


Из 7126 генов, хоть как-то покрыто 6273 из них. То есть 95% процента кодирующей части покрыто, но при этом ничего не известно про покрытие, вполне может быть что среднее покрытие равно 1. 

### 3. Сбока de novo

Запуск Trinity

In [ ]:
! /Molly/chernikova/soft/Trinity/Trinity --seqType fq --left /Johnny/students/NGS/data/8/SRR453566_1.fastq --right /Johnny/students/NGS/data/8/SRR453566_2.fastq --CPU 6 --max_memory 20G

Запуск rnaQUAST

In [ ]:
! python2 /Molly/chernikova/soft/rnaQUAST-1.5.1/rnaQUAST.py --transcripts trinity_out_dir/Trinity.fasta --reference /Johnny/students/NGS/data/8/ref.fa --gtf /Johnny/students/NGS/data/8/genes.gtf -o ./out/ 

Из 7126 генов на 95% покрыто только 3909 генов(55%).  